In [342]:
# !pip install talos
import talos as ta

import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import tensorflow as tf

import keras
from keras.layers import *
from keras.callbacks import *
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import LeakyReLU

In [343]:
target = pd.read_csv('y_train.csv')
data = pd.read_csv('x_train.csv')
test = pd.read_csv('x_test.csv')
sub = pd.read_csv('sample_submission.csv')
print('Data is ready !!')

Data is ready !!


In [344]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

In [345]:
def fe_step0 (actual):
    
    # https://www.mathworks.com/help/aeroblks/quaternionnorm.html
    # https://www.mathworks.com/help/aeroblks/quaternionmodulus.html
    # https://www.mathworks.com/help/aeroblks/quaternionnormalize.html
        
    actual['norm_quat'] = (actual['orientation_X']**2 + actual['orientation_Y']**2 + actual['orientation_Z']**2 + actual['orientation_W']**2)
    actual['mod_quat'] = (actual['norm_quat'])**0.5
    actual['norm_X'] = actual['orientation_X'] / actual['mod_quat']
    actual['norm_Y'] = actual['orientation_Y'] / actual['mod_quat']
    actual['norm_Z'] = actual['orientation_Z'] / actual['mod_quat']
    actual['norm_W'] = actual['orientation_W'] / actual['mod_quat']
    
    return actual

In [346]:
data = fe_step0(data)
test = fe_step0(test)
print(data.shape)
data.head()

(487680, 19)


,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,norm_quat,mod_quat,norm_X,norm_Y,norm_Z,norm_W
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532,0.999997,0.999999,-0.758531,-0.634351,-0.10488,-0.10597
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128,0.999995,0.999998,-0.758532,-0.634342,-0.10490,-0.10600
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267,1.000006,1.000003,-0.758528,-0.634348,-0.10492,-0.10597
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960,1.000009,1.000005,-0.758516,-0.634357,-0.10495,-0.10597
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410,0.999995,0.999997,-0.758522,-0.634352,-0.10495,-0.10596


In [347]:
def fe_step1 (actual):
    """Quaternions to Euler Angles"""
    
    x, y, z, w = actual['norm_X'].tolist(), actual['norm_Y'].tolist(), actual['norm_Z'].tolist(), actual['norm_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    return actual

In [348]:
data = fe_step1(data)
test = fe_step1(test)
print (data.shape)
data.head()

(487680, 22)


,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,linear_acceleration_Z,norm_quat,mod_quat,norm_X,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,...,-9.7532,0.999997,0.999999,-0.758531,-0.634351,-0.10488,-0.10597,2.843273,-0.024668,1.396667
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,...,-9.4128,0.999995,0.999998,-0.758532,-0.634342,-0.10490,-0.10600,2.843201,-0.024662,1.396651
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,...,-8.7267,1.000006,1.000003,-0.758528,-0.634348,-0.10492,-0.10597,2.843222,-0.024728,1.396677
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,...,-10.0960,1.000009,1.000005,-0.758516,-0.634357,-0.10495,-0.10597,2.843183,-0.024769,1.396712
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,...,-10.4410,0.999995,0.999997,-0.758522,-0.634352,-0.10495,-0.10596,2.843197,-0.024785,1.396698


In [349]:
def feat_eng(data):
    
    df = pd.DataFrame()
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 + data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 + data['linear_acceleration_Z']**2)**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 + data['orientation_Z']**2)**0.5
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    
    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_mean_change_of_abs_change'] = data.groupby('series_id')[col].apply(mean_change_of_abs_change)
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
    return df

In [350]:
del data['row_id']
del test['row_id']
del data['measurement_number']
del test['measurement_number']

In [351]:
%%time
data = feat_eng(data)
test = feat_eng(test)
print ("New features: ",data.shape)

New features:  (3810, 276)
Wall time: 2min 40s


In [352]:
data.fillna(0,inplace=True)
test.fillna(0,inplace=True)
data.replace(-np.inf,0,inplace=True)
data.replace(np.inf,0,inplace=True)
test.replace(-np.inf,0,inplace=True)
test.replace(np.inf,0,inplace=True)

In [353]:
data.head()

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,...,-0.634351,-0.10488,-0.10597,2.843273,-0.024668,1.396667,0.109076,10.005392,0.994368,91.728917
1,0,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,...,-0.634342,-0.10490,-0.10600,2.843201,-0.024662,1.396651,0.074240,9.538638,0.994364,128.483970
2,0,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,...,-0.634348,-0.10492,-0.10597,2.843222,-0.024728,1.396677,0.030428,8.874697,0.994372,291.666195
3,0,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,...,-0.634357,-0.10495,-0.10597,2.843183,-0.024769,1.396712,0.025082,10.164638,0.994374,405.249042
4,0,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,...,-0.634352,-0.10495,-0.10596,2.843197,-0.024785,1.396698,0.010603,10.556132,0.994368,995.561076


In [354]:
test.head()

,series_id,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,...,-0.988644,-0.148011,0.003350,2.844735,-0.014254,3.091598,0.028177,6.959611,0.999990,247.000018
1,0,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,...,-0.988624,-0.148161,0.003439,2.844435,-0.014411,3.091804,0.153987,8.992450,0.999990,58.397648
2,0,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,...,-0.988610,-0.148260,0.003571,2.844239,-0.014658,3.091976,0.145114,11.619024,0.999993,80.068203
3,0,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,...,-0.988625,-0.148171,0.003609,2.844420,-0.014712,3.092086,0.085687,12.387813,0.999989,144.570382
4,0,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,...,-0.988663,-0.147920,0.003477,2.844920,-0.014434,3.092079,0.155550,11.300393,0.999991,72.648208


In [355]:
data = target.merge(data, on='series_id')

In [356]:
target.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [357]:
encode_dic = {'fine_concrete': 0, 
              'concrete': 1, 
              'soft_tiles': 2, 
              'tiled': 3, 
              'soft_pvc': 4,
              'hard_tiles_large_space': 5, 
              'carpet': 6, 
              'hard_tiles': 7, 
              'wood': 8}

In [358]:
decode_dic = {0: 'fine_concrete',
              1: 'concrete',
              2: 'soft_tiles',
              3: 'tiled',
              4: 'soft_pvc',
              5: 'hard_tiles_large_space',
              6: 'carpet',
              7: 'hard_tiles',
              8: 'wood'}

In [359]:
data['surface'] = data['surface'].map(encode_dic).astype(int)
target['surface'] = target['surface'].map(encode_dic).astype(int)

In [360]:
print(len(data))
data.head()

487680


,series_id,group_id,surface,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
0,0,13,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,...,-0.634351,-0.10488,-0.10597,2.843273,-0.024668,1.396667,0.109076,10.005392,0.994368,91.728917
1,0,13,0,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,...,-0.634342,-0.10490,-0.10600,2.843201,-0.024662,1.396651,0.074240,9.538638,0.994364,128.483970
2,0,13,0,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,...,-0.634348,-0.10492,-0.10597,2.843222,-0.024728,1.396677,0.030428,8.874697,0.994372,291.666195
3,0,13,0,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,...,-0.634357,-0.10495,-0.10597,2.843183,-0.024769,1.396712,0.025082,10.164638,0.994374,405.249042
4,0,13,0,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,...,-0.634352,-0.10495,-0.10596,2.843197,-0.024785,1.396698,0.010603,10.556132,0.994368,995.561076


In [361]:
data.groupby(['group_id']).agg(len).head()

,series_id,surface,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
group_id,,,,,,,,,,,,,,,,,,,,,
0,7296,7296,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,...,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0
1,4864,4864,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,...,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0,4864.0
2,2304,2304,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,...,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0,2304.0
3,7296,7296,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,...,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0
4,7296,7296,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,...,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0


In [362]:
num = [68, 72, 61, 62, 63, 22, 17, 71, 66, 70, 69, 58, 59, 44, 60, 52, 67, 65, 49]
nums = [1671, 1610, 1455, 1454, 1399]
ht_vd = data[data['surface']==7]
ht = ht_vd[ht_vd.series_id.isin(nums)]
vdata = data[data.group_id.isin(num)]
vdata = pd.concat((vdata, ht))
vdata = vdata.sort_values('series_id')

tar = target[target.series_id.isin(nums)]
vtarget = target[target.group_id.isin(num)]
vtarget = pd.concat((vtarget, tar))
vtarget = vtarget.sort_values('series_id')

In [363]:
vtarget.head()

,series_id,group_id,surface
4,4,22,2
22,22,22,2
55,55,22,2
59,59,22,2
95,95,22,2


In [364]:
vdata.groupby(['group_id']).agg(len).head()

,series_id,surface,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
group_id,,,,,,,,,,,,,,,,,,,,,
17,1536,1536,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,...,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0,1536.0
22,7296,7296,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,...,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0,7296.0
27,640,640,640.0,640.0,640.0,640.0,640.0,640.0,640.0,640.0,...,640.0,640.0,640.0,640.0,640.0,640.0,640.0,640.0,640.0,640.0
44,9088,9088,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,...,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0,9088.0
49,7680,7680,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,...,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0,7680.0


In [365]:
data =(pd.merge(data,vdata,indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

In [366]:
target =(pd.merge(target,vtarget,indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

In [367]:
target.head()

,series_id,group_id,surface
0,0,13,0
1,1,31,1
2,2,20,1
3,3,31,1
5,5,1,3


In [368]:
from imblearn.over_sampling import SMOTE, ADASYN
sm = SMOTE(random_state=2)
ada = ADASYN(random_state=42)

In [369]:
# data_sm, lab_sm = sm.fit_sample(co_d,co_t)
# df_sm = pd.DataFrame(data_sm, columns=data.columns.values)
# t_sm = df_sm['surface']

In [370]:
data.head()

,series_id,group_id,surface,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,...,norm_Y,norm_Z,norm_W,euler_x,euler_y,euler_z,totl_anglr_vel,totl_linr_acc,totl_xyz,acc_vs_vel
0,0,13,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,...,-0.634351,-0.10488,-0.10597,2.843273,-0.024668,1.396667,0.109076,10.005392,0.994368,91.728917
1,0,13,0,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,...,-0.634342,-0.10490,-0.10600,2.843201,-0.024662,1.396651,0.074240,9.538638,0.994364,128.483970
2,0,13,0,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,...,-0.634348,-0.10492,-0.10597,2.843222,-0.024728,1.396677,0.030428,8.874697,0.994372,291.666195
3,0,13,0,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,...,-0.634357,-0.10495,-0.10597,2.843183,-0.024769,1.396712,0.025082,10.164638,0.994374,405.249042
4,0,13,0,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,...,-0.634352,-0.10495,-0.10596,2.843197,-0.024785,1.396698,0.010603,10.556132,0.994368,995.561076


In [371]:
btarget = data['surface']
btarget.head()

0    0
1    0
2    0
3    0
4    0
Name: surface, dtype: int32

In [372]:
bvtarget = vdata['surface']
bvtarget.head()

512    2
605    2
604    2
603    2
602    2
Name: surface, dtype: int32

In [373]:
del data['series_id']
del data['surface']
del data['group_id']
del vdata['series_id']
del vdata['surface']
del vdata['group_id']
del test['series_id']

In [374]:
print(data.shape)
print(vdata.shape)
test.shape

(358912, 23)
(128768, 23)


(488448, 23)

In [375]:
data.to_csv('data.csv', index=False)

In [376]:
vdata.to_csv('vdata.csv', index=False)

In [377]:
target.to_csv('target.csv', index=False)

In [378]:
btarget.to_csv('btarget.csv', index=False)

In [379]:
vtarget.to_csv('vtarget.csv', index=False)

In [383]:
bvtarget.to_csv('bvtarget.csv', index=False)

In [380]:
test.to_csv('test.csv', index=False)

In [385]:
bvtarget

512       2
605       2
604       2
603       2
602       2
601       2
600       2
599       2
598       2
597       2
596       2
595       2
594       2
593       2
606       2
592       2
590       2
589       2
588       2
587       2
586       2
585       2
584       2
583       2
582       2
581       2
580       2
579       2
578       2
591       2
         ..
487236    8
487237    8
487238    8
487239    8
487240    8
487241    8
487242    8
487243    8
487244    8
487245    8
487246    8
487247    8
487248    8
487249    8
487250    8
487251    8
487252    8
487253    8
487254    8
487255    8
487256    8
487257    8
487258    8
487259    8
487260    8
487261    8
487262    8
487263    8
487233    8
487169    8
Name: surface, Length: 128768, dtype: int32

In [223]:
data = data.values
vdata = vdata.values
test = test.values
target = target['surface']
vtarget = vtarget['surface']

In [384]:
print(target.shape)
print(vtarget.shape)
print(btarget.shape)
print(bvtarget.shape)
print(vdata.shape)
print(data.shape)
test.shape

(2804, 3)
(1006, 3)
(358912,)
(128768,)
(128768, 23)
(358912, 23)


(488448, 23)

In [33]:
# data = data.reshape((2804, 128, 23))
# vdata = vdata.reshape((1006, 128, 23))
# test = test.reshape((3816, 128, 23))

In [ ]:
data = data.reshape((358912, 23))
vdata = vdata.reshape((128768, 23))
test = test.reshape((488448, 23))

In [34]:
print(vdata.shape)
print(data.shape)
test.shape

(1006, 128, 23)
(2804, 128, 23)


(3816, 128, 23)

In [226]:
model = Sequential()

# data = data.reshape((2804, 128, 23))
# vdata = vdata.reshape((1006, 128, 23))
# test = test.reshape((3816, 128, 23))
num_classes = 9


# model.add(Conv1D(16, (3,), padding='same', input_shape=data.shape[1:], use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv1D(16, (3,), use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(MaxPooling1D(pool_size=(2,)))
# # model.add(SpatialDropout1D(0.25))

# model.add(Conv1D(64, (3,), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(Conv1D(64, (3,), use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
# model.add(MaxPooling1D(pool_size=(2,)))
# # model.add(SpatialDropout1D(0.25))


# model.add(Flatten())
# model.add(LSTM(128, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(BatchNormalization())
# model.add(Activation('tanh'))
model.add(Dense(64, input_dim=23, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(64, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(64, use_bias=False, kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [227]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
# Let's train the model using RMSprop
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# loss='mearn_squared_error'
# 'adam'
# 'Nadam'
# loss = kullback_leibler_divergence
# 

In [240]:
model.fit(data, btarget,
              batch_size=128,
              epochs=100,
              validation_data=(vdata, bvtarget),
              shuffle=True)

Train on 358912 samples, validate on 128768 samples
Epoch 1/100
358912/358912 [==============================] - 20s 56us/step - loss: 0.3971 - acc: 0.8774 - val_loss: 3.4897 - val_acc: 0.4373
Epoch 2/100
358912/358912 [==============================] - 19s 53us/step - loss: 0.3978 - acc: 0.8767 - val_loss: 3.4705 - val_acc: 0.4420
Epoch 3/100
358912/358912 [==============================] - 18s 51us/step - loss: 0.3955 - acc: 0.8776 - val_loss: 3.3394 - val_acc: 0.4376
Epoch 4/100
358912/358912 [==============================] - 18s 51us/step - loss: 0.3974 - acc: 0.8770 - val_loss: 3.3886 - val_acc: 0.4544
Epoch 5/100
358912/358912 [==============================] - 19s 52us/step - loss: 0.3924 - acc: 0.8794 - val_loss: 3.4657 - val_acc: 0.4341
Epoch 6/100
358912/358912 [==============================] - 19s 52us/step - loss: 0.4002 - acc: 0.8767 - val_loss: 3.1260 - val_acc: 0.4548
Epoch 7/100
358912/358912 [==============================] - 19s 52us/step - loss: 0.3977 - acc: 0.877

358912/358912 [==============================] - 19s 52us/step - loss: 0.3883 - acc: 0.8805 - val_loss: 3.5824 - val_acc: 0.4385
Epoch 59/100
358912/358912 [==============================] - 19s 54us/step - loss: 0.3897 - acc: 0.8794 - val_loss: 3.5666 - val_acc: 0.4347
Epoch 60/100
358912/358912 [==============================] - 19s 53us/step - loss: 0.3889 - acc: 0.8800 - val_loss: 3.4759 - val_acc: 0.4303
Epoch 61/100
358912/358912 [==============================] - 19s 52us/step - loss: 0.3896 - acc: 0.8794 - val_loss: 3.4084 - val_acc: 0.4649
Epoch 62/100
358912/358912 [==============================] - 19s 52us/step - loss: 0.3900 - acc: 0.8791 - val_loss: 3.4439 - val_acc: 0.4694
Epoch 63/100
358912/358912 [==============================] - 18s 51us/step - loss: 0.3901 - acc: 0.8798 - val_loss: 3.5558 - val_acc: 0.4371
Epoch 64/100
358912/358912 [==============================] - 19s 53us/step - loss: 0.3882 - acc: 0.8800 - val_loss: 3.3676 - val_acc: 0.4604
Epoch 65/100
358912

In [260]:
preds = model.predict_classes(test)
preds = preds[0:-1:128]
sub['surface'] = preds
sub['surface'] = sub['surface'].map(decode_dic)
sub.head(15)

,series_id,surface
0,0,tiled
1,1,concrete
2,2,tiled
3,3,wood
4,4,soft_pvc
5,5,concrete
6,6,concrete
7,7,concrete
8,8,carpet
9,9,carpet


In [261]:
sub.to_csv('submission10.csv', index=False)

In [280]:
btarget = btarget.values

In [283]:
bvtarget = bvtarget.values

In [336]:
p = {'activation':['tanh', 'relu'],
         'last_activation': ['softmax'],
         'optimizer': ['Nadam', 'Adam', 'RMSprop'],
         'losses': ['sparse_categorical_crossentropy'],
         'first_neuron':[8, 16, 32, 64, 128],
         'hidden_layers':[0, 1, 2],
         'kernel_initializer': ['uniform','normal'],
         'batch_size': [10,20,30,40,60,100],
         'dropout': (0, 0.5, 10),
         'shapes':['brick','long_funnel'],
         'weight_regulizer': [None],
         'epochs': [200]}

In [339]:

def kaggle_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
        
    model.add(Dense(params['first_neuron'], input_dim=23, activation=params['activation'], kernel_initializer=params['kernel_initializer']))
    model.add(Dropout(params['dropout']))
        
    ta.model.hidden_layers(model, params, 1)
        
    model.add(Dense(9, activation=params['last_activation'], kernel_initializer=params['kernel_initializer']))
        
    model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['acc'])

    out = model.fit(data, btarget,
                         batch_size=params['batch_size'],
                         epochs=params['epochs'],
                         validation_data=(vdata, bvtarget),
                         verbose=0)

    return out, model

    


In [340]:
scan_object = ta.Scan(data, btarget, model=kaggle_model, params=p, grid_downsample=0.1)





















  0%|                          | 0/2160 [00:00<?, ?it/s]



















  0%|           | 1/2160 [35:06<1263:34:08, 2106.92s/it]



















  0%|         | 2/2160 [1:27:59<1454:36:15, 2426.59s/it]



















  0%|         | 3/2160 [3:15:47<2180:22:20, 3639.01s/it]

KeyboardInterrupt: 

In [341]:
scan_object

NameError: name 'scan_object' is not defined

In [40]:
# sub073 = pd.read_csv('../input/robots-best-submission/mybest0.73.csv')
# sub072 = pd.read_csv('../input/robots-best-submission/sub_0.72.csv')
# sub072_2 = pd.read_csv('../input/robots-best-submission/sub_0.72_2.csv')
# sub071 = pd.read_csv('../input/robots-best-submission/sub_0.71.csv')
# sub06 = pd.read_csv('../input/robots-best-submission/sub_0.6.csv')

# sub073 = sub073.rename(columns = {'surface':'surface073'})
# sub072 = sub072.rename(columns = {'surface':'surface072'})
# sub072_2 = sub072_2.rename(columns = {'surface':'surface072_2'})
# sub071 = sub071.rename(columns = {'surface':'surface071'})
# sub06 = sub06.rename(columns = {'surface':'surface06'})
# print ("Submission data is ready")

In [41]:
# subtest = pd.concat([sub073['series_id'], sub073['surface073'], sub072['surface072'], sub071['surface071'], sub06['surface06']], axis=1)
# subtest.head()

In [42]:
# differents = []
# for i in range (0,subtest.shape[0]): 
#     labels = list(subtest.iloc[i,1:])
#     result = len(set(labels))>1
#     if result:
#         differents.append((i, str(labels)))
        
# differents = pd.DataFrame(differents, columns=['idx','group']) 
# differents.head()

In [43]:
# differents['group'].nunique()

In [44]:
# differents['count'] = differents.groupby('group')['group'].transform('count')
# differents = differents.sort_values(by=['count'], ascending=False)
# differents = differents.drop(['idx'],axis=1)
# differents = differents.drop_duplicates()

In [45]:
# differents.head(10)

In [46]:
# fc_vd = data[data['surface']==0]
# co_vd = data[data['surface']==1]
# st_vd = data[data['surface']==2]
# ti_vd = data[data['surface']==3]
# sp_vd = data[data['surface']==4]
# htls_vd = data[data['surface']==5]
# ca_vd = data[data['surface']==6]
# ht_vd = data[data['surface']==7]
# wo_vd = data[data['surface']==8]

In [47]:
# nfc_vd = data[data['surface']!=0]
# nco_vd = data[data['surface']!=1]
# nst_vd = data[data['surface']!=2]
# nti_vd = data[data['surface']!=3]
# nsp_vd = data[data['surface']!=4]
# nhtls_vd = data[data['surface']!=5]
# nca_vd = data[data['surface']!=6]
# nht_vd = data[data['surface']!=7]
# nwo_vd = data[data['surface']!=8]

In [48]:
# nfc_vd['surface']=1
# nco_vd['surface']=0
# nst_vd['surface']=0
# nti_vd['surface']=0
# nsp_vd['surface']=0
# nhtls_vd['surface']=0
# nca_vd['surface']=0
# nht_vd['surface']=0
# nwo_vd['surface']=0

In [49]:
# fc_d = pd.concat((fc_vd, nfc_vd))
# co_d = pd.concat((co_vd, nco_vd))
# st_d = pd.concat((st_vd, nst_vd))
# ti_d = pd.concat((ti_vd, nti_vd))
# sp_d = pd.concat((sp_vd, nsp_vd))
# htls_d = pd.concat((htls_vd, nhtls_vd))
# ca_d = pd.concat((ca_vd, nca_vd))
# ht_d = pd.concat((ht_vd, nht_vd))
# wo_d = pd.concat((wo_vd, nwo_vd))

In [50]:
# fc_t = fc_d['surface']
# co_t = co_d['surface']
# st_t = st_d['surface']
# ti_t = ti_d['surface']
# sp_t = sp_d['surface']
# htls_t = htls_d['surface']
# ca_t = ca_d['surface']
# ht_t = ht_d['surface']
# wo_t = wo_d['surface']

In [51]:
# fc_d = fc_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# co_d = co_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# st_d = st_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# ti_d = ti_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# sp_d = sp_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# htls_d = htls_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# ca_d = ca_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# ht_d = ht_d.drop(['series_id', 'group_id', 'surface'], axis=1)
# wo_d = wo_d.drop(['series_id', 'group_id', 'surface'], axis=1)